# Dependencias:

In [ ]:
#from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml.html as html
import os
import datetime

#'V_Title'
web_URL="https://www.larepublica.co"

# Codigo

Prueba de conexion.

In [ ]:
try:
  response=requests.get(web_URL)
  if response.status_code==200:
    print(f"Conexion funcionando (value): {response.status_code}")
  else:
    raise error(print(f"Conexion no funcionando (value): {response.status_code}"))
except ValueError as error:
  print(error)

Conexion funcionando (value): 200


Captura del html

In [ ]:
home=html.fromstring(response.content.decode("utf-8"))
home

<Element html at 0x7fa934efbf50>

## 1- Elementos que sirver para buscar desde la pagina al html (mashes)

### Contains(type, value)

    Xpath=//tagname[contains (@Attribute, ‘Value’)]

In [ ]:
#contains(text, value)
contains="//*[contains(text(),'Las')]"
contains_word=home.xpath(contains)
#contain retorna los nodos que contiene la informacion pasada
contains_word

[<Element a at 0x7fa934efc410>,
 <Element a at 0x7fa934efc170>,
 <Element a at 0x7fa934efc350>,
 <Element p at 0x7fa934efc650>,
 <Element a at 0x7fa934efc6b0>]

In [ ]:
#contains(href, value)
contains="//*[contains(@href,'www')]"
contains_word=home.xpath(contains)
#contain retorna los nodos con atributos href que contienen la informacion pasada
contains_word[:7]

[<Element link at 0x7fa934efc950>,
 <Element a at 0x7fa934efca10>,
 <Element a at 0x7fa934efc7d0>,
 <Element a at 0x7fa934efcbf0>,
 <Element a at 0x7fa934efcc50>,
 <Element a at 0x7fa934efccb0>,
 <Element a at 0x7fa934efcd10>]

In [ ]:
#contains(href, value) and contains(text, value)
contains="//*[contains(@href,'www') and contains(text(),'Las')]"
contains_word=home.xpath(contains)
#contain retorna los nodos con atributos href y text que contienen la informacion pasada pudiendo encontrar textos con enlaces.
contains_word

[<Element a at 0x7fa934efc410>,
 <Element a at 0x7fa934efc170>,
 <Element a at 0x7fa934efc350>,
 <Element a at 0x7fa934efc6b0>]

In [ ]:
#contains(href, value) and contains(text, value) return atributos
contains="//*[contains(@href,'www') and contains(text(),'Las')]/@*"
contains_word=home.xpath(contains)
#podemos ver los atributos que contine el nodo(a) pero no cual atributo es, 
#como ya de filtro por href podemos asumir que el link es el href
contains_word

['https://www.larepublica.co/globoeconomia/las-muchas-formas-ocultas-que-utilizan-empresas-del-mundo-para-subir-los-precios-3301997',
 'globoeconomiaSect',
 'https://www.larepublica.co/ocio/hacia-asia-las-mejores-universidades-de-asia-3299180',
 'ocioSect',
 'https://www.larepublica.co/globoeconomia/como-son-las-casas-ataud-salen-a-us225-por-mes-y-ya-viven-mas-de-200000-personas-3301010',
 'globoeconomiaSect',
 'https://www.larepublica.co/ocio/conozca-las-canciones-esenciales-para-recordar-la-obra-del-maestro-julio-erazo-3302101',
 'ocioSect']

In [ ]:
#contains(href, value) and contains(text, value) /.. caes sobre los mismos nodos /../.. caes en los padres
contains="//*[contains(@href,'www') and contains(text(),'Las')]/../../*"
contains_word=home.xpath(contains)
#podemos ver que los nodos padres son varios el ojetivo es filtra por estos para obtener solo los links a las noticias.
contains_word

[<Element a at 0x7fa934efc1d0>,
 <Element text-fill at 0x7fa934efc290>,
 <Element a at 0x7fa934efc050>,
 <Element div at 0x7fa934efca70>,
 <Element a at 0x7fa934f12050>,
 <Element span at 0x7fa934f120b0>,
 <Element text-fill at 0x7fa934f12110>,
 <Element a at 0x7fa934f12170>,
 <Element text-fill at 0x7fa934f121d0>,
 <Element a at 0x7fa934f12230>,
 <Element span at 0x7fa934f12290>,
 <Element text-fill at 0x7fa934f12650>]

#### Filtrado por nodo

In [ ]:
#contains(href, value) and contains(text, '') /text-fill 
contains="//text-fill/a[contains(@href,'www') and contains(text(),'')]/@*"
contains_word=home.xpath(contains)
#para filtral todos lo text los igualamos a ''.
#con text-fill tenemos buenos resultados
contains_word[:6], len(contains_word)

(['https://www.larepublica.co/ocio/en-el-dia-mundial-de-la-radio-conozca-su-historia-desde-el-transistor-hasta-el-ipod-3302224',
  'ocioSect',
  'https://www.larepublica.co/ocio/participacion-en-eventos-de-talla-mundial-y-talento-creativo-favorecen-al-cine-nacional-3302021',
  'ocioSect',
  'https://www.larepublica.co/ocio/conozca-los-costos-de-produccion-de-las-peliculas-nominadas-a-los-premios-oscar-3302076',
  'ocioSect'],
 143)

In [ ]:
#contains(href, value) and contains(text, '') /span 
contains="//span/a[contains(@href,'www') and contains(text(),'')]/@*"
contains_word=home.xpath(contains)
#para filtral todos lo text los igualamos a ''.
#con span retorna los links a los podcats.
contains_word, len(contains_word)

(['https://www.larepublica.co/podcast/voces-para-educar',
  'nameShow',
  'https://www.larepublica.co/podcast/protagonistas-de-la-economia-colombiana',
  'nameShow',
  'https://www.larepublica.co/podcast/lr-radio',
  'nameShow'],
 6)

In [ ]:
#contains(href, value) and contains(text, '') /div 
contains="//div/a[contains(@href,'www') and contains(text(),'')]/@*"
contains_word=home.xpath(contains)
#para filtral todos lo text los igualamos a ''.
#con div tenemos las secciones de noticias
contains_word[:6], len(contains_word)

(['https://www.larepublica.co/finanzas',
  'https://www.larepublica.co/economia',
  'https://www.larepublica.co/empresas',
  'https://www.larepublica.co/ocio',
  'https://www.larepublica.co/globoeconomia',
  'https://www.agronegocios.co'],
 352)

In [ ]:
#contains(href, value) and contains(text, '') /a 
contains="//a/a[contains(@href,'www') and contains(text(),'')]/node()"
contains_word=home.xpath(contains)
#un tag a nio contiene un tag a
contains_word[:6], len(contains_word)

([], 0)

## 2- Aplicar lo mismo para hallar el titulo, el resumen y la noticia

- el titulo no tiene enlase a link y esta dentro de un nodo h2 o h3
- el resumen no tiene enlase a link y esta dentro de un nodo h menor a 2
- la noticia no tiene enlase a link y esta dentro de un nodo p y este dentro de un div


Capturamos los links con los filtros anteriores

In [ ]:
#contains(href, value) and contains(text, '') /text-fill / href
contains="//text-fill/a[contains(@href,'www') and contains(text(),'')]/@href"
links=home.xpath(contains)
links[:6], len(links)

(['https://www.larepublica.co/ocio/en-el-dia-mundial-de-la-radio-conozca-su-historia-desde-el-transistor-hasta-el-ipod-3302224',
  'https://www.larepublica.co/ocio/participacion-en-eventos-de-talla-mundial-y-talento-creativo-favorecen-al-cine-nacional-3302021',
  'https://www.larepublica.co/ocio/conozca-los-costos-de-produccion-de-las-peliculas-nominadas-a-los-premios-oscar-3302076',
  'https://www.larepublica.co/ocio/equipos-de-europa-han-ganado-la-copa-mundial-de-clubes-14-veces-ultimos-17-anos-3302023',
  'https://www.larepublica.co/ocio/asistentes-del-julio-mario-santo-domingo-han-aumentado-30-con-menos-restricciones-3302082',
  'https://www.larepublica.co/ocio/con-el-evento-de-taco-fest-triplicamos-el-empleo-de-toda-la-cadena-gastronomica-3302089'],
 75)

Obtener html

In [ ]:
notice= links[4]
notice_response=requests.get(notice)
notice_home=html.fromstring(notice_response.content.decode("utf-8"))
notice_response

<Response [200]>

### Aplicar *Xpath*

#### Titulo

In [ ]:
#para ver el titulo que buscamos podemos revisar el link
text=notice.replace("-"," ")
notice

'https://www.larepublica.co/ocio/asistentes-del-julio-mario-santo-domingo-han-aumentado-30-con-menos-restricciones-3302082'

In [ ]:
#not(contains(@href,'www')) and contains(text(),'30') and not(contains(text(),'\n')
#con las condiciones de arriba nos quitamos de encima cualquier otro texto
contains="//*[not(contains(@href,'www')) and contains(text(),'30') and not(contains(text(),'\n'))]/text()"
#con la palabra clave "30" podemos buscar los textos que concuerden con el titulo
contains_word=notice_home.xpath(contains)
contains_word, len(contains_word)

(['Asistentes del Santo Domingo subieron 30% con menos restricciones',
  'Asistentes del Santo Domingo subieron 30% con menos restricciones',
  'Asistentes del Santo Domingo subieron 30% con menos restricciones',
  'El teatro cuenta con un calendario de más de 30 eventos para los próximos meses, que incluyen música, danza y festivales ',
  'Los planes culturales volvieron a la presencialidad con fuerza. En el caso del Teatro Mayor Julio Mario Santo Domingo, se ha traducido en un incremento de más de 30% en los asistentes desde que se levantaron las restricciones de movilidad.',
  'La agenda está enriquecida con más de 30 eventos entre lo que resta de febrero y abril, que incluyen, música, danza y por supuesto, teatro. Con los que se espera aumentar aún más la asistencia de los espectadores.',
  'Asistentes del Santo Domingo subieron 30% con menos restricciones'],
 7)

In [ ]:
#not(contains(@href,'www')) and contains(text(),'30') and not(contains(text(),'\n') /parent::*

contains="//*[not(contains(@href,'www')) and contains(text(),'30') and not(contains(text(),'\n'))]/parent::*"
#con "parent::*" conseguimos los nodos padres de los textos
contains_word=notice_home.xpath(contains)
contains_word, len(contains_word)

([<Element head at 0x7fa934e9ba70>,
  <Element h1 at 0x7fa934e9b9b0>,
  <Element text-fill at 0x7fa934e9bc50>,
  <Element div at 0x7fa934e9bcb0>,
  <Element div at 0x7fa934e9bd10>,
  <Element div at 0x7fa934e9bd70>],
 6)

In [ ]:
# /title contains(href, value) and contains(text, '')

contains="//title[not(contains(@href,'www')) and contains(text(),'') and not(contains(text(),'\n'))]/text()"
#ahora podemos  filtrar cada titulo de cada link
contains_word=notice_home.xpath(contains)
contains_word, len(contains_word)

(['Asistentes del Santo Domingo subieron 30% con menos restricciones'], 1)

#### Summary

In [ ]:
#not(contains(@href,'www')) and contains(text(),'calendario de más de 30') and not(contains(text(),'\n')
#Lo mejor es revisar el link y aplicar un texto clave, pARA CREAR UN FILTRO
contains="//body//*[not(contains(@href,'www')) and contains(text(),'calendario de más de 30') and not(contains(text(),'\n'))]/text()"
#SABEMOS QUE EL TEXTO DEBE ESTA EN BODY.
contains_word=notice_home.xpath(contains)
contains_word, len(contains_word)

(['El teatro cuenta con un calendario de más de 30 eventos para los próximos meses, que incluyen música, danza y festivales '],
 1)

In [ ]:
contains="//body//*[not(contains(@href,.)) and contains(text(),'calendario de más de 30') and not(contains(text(),'\n'))]"
#vemos que el tag es un p
contains_word=notice_home.xpath(contains)
len(contains_word), contains_word[:10]

(1, [<Element p at 0x7fa934e947d0>])

In [ ]:
contains="//body//p[not(contains(@href,.)) and contains(text(),'calendario de más de 30') and not(contains(text(),'\n'))]/ancestor::*"
#vemos que el tag ancestros son un div
contains_word=notice_home.xpath(contains)
len(contains_word), contains_word[:10]

(9,
 [<Element html at 0x7fa934ef93b0>,
  <Element body at 0x7fa934ea2470>,
  <Element div at 0x7fa934ea2230>,
  <Element div at 0x7fa934ea24d0>,
  <Element div at 0x7fa934ea2830>,
  <Element div at 0x7fa934ea2770>,
  <Element div at 0x7fa934ea27d0>,
  <Element div at 0x7fa934ea2890>,
  <Element div at 0x7fa934e9bcb0>])

In [ ]:
contains="//body//p[not(contains(@href,.)) and contains(text(),'calendario de más de 30') and not(contains(text(),'\n'))]/parent::*/@*"
#el tag padre es un div con un atributo de valor lead
contains_word=notice_home.xpath(contains)
len(contains_word), contains_word[:10]

(1, ['lead'])

In [ ]:
contains="//body//div[contains(@*,'lead')]/p/text()"
#ya con esto podemos montar un filtro xpath generar
summary=notice_home.xpath(contains)
len(summary), summary[:10]

(1,
 ['Las acciones de la matriz de PedidosYa cayeron ayer tras conocerse proyecciones débiles de las ganancias para este año'])

#### contenido

In [ ]:
contains="//body//div[not(contains(@*,'lead'))]/p/text()"
#buscamos el text y omitimos el resumen
summary=notice_home.xpath(contains)
len(summary), summary[:10]

(12,
 ['Los planes culturales volvieron a la presencialidad con fuerza. En el caso del Teatro Mayor Julio Mario Santo Domingo, se ha traducido en un incremento de más de 30% en los asistentes desde que se levantaron las restricciones de movilidad.',
  'La agenda está enriquecida con más de 30 eventos entre lo que resta de febrero y abril, que incluyen, música, danza y por supuesto, teatro. Con los que se espera aumentar aún más la asistencia de los espectadores.',
  '"La pandemia es un ejemplo de cómo las audiencias se volcaron a la oferta cultural digital, llegando a números que en su momento eran inimaginables. Esperamos que ahora ese consumo se vea reflejado en lo presencial, pues el público que compra entradas para los eventos culturales es un patrocinador fundamental", destacó Ramiro Osorio, director del Teatro Mayor Julio Mario Santo Domingo.',
  'Para el resto del año, también destacan las presentaciones de artistas como Ana Torraja el 12 de mayo; Don Tetto el 4 de junio; Puerto

In [ ]:
contains="//body//div[not(contains(@*,'lead'))]/p/parent::*"
#revisamos los nodos p y sus padres
summary=notice_home.xpath(contains)
len(summary), summary[:10]

(10,
 [<Element div at 0x7fa934d9c770>,
  <Element div at 0x7fa934d9c7d0>,
  <Element div at 0x7fa934d9c830>,
  <Element div at 0x7fa934d9c890>,
  <Element div at 0x7fa934d9c8f0>,
  <Element div at 0x7fa934d9c950>,
  <Element div at 0x7fa934d9c9b0>,
  <Element div at 0x7fa934d9ca10>,
  <Element div at 0x7fa934d9ca70>,
  <Element div at 0x7fa934d9cad0>])

In [ ]:
contains="//body//div[not(contains(@*,'lead'))]/p/parent::*/@class"
#revisamos los atributosde los padres
summary=notice_home.xpath(contains)
len(summary), summary[:10]

(9,
 ['html-content',
  'col-7 pl-3 pr-3',
  'col-7 pl-3 pr-3',
  'col-7 pl-3 pr-3',
  'info',
  'col-2 pl-0 pr-0 mr-5',
  'col-5 menu-first mr-6',
  'flex-grow-1 menu-second mr-6',
  'portales-aliados mt-5 pt-3'])

In [ ]:
contains="//body//div[not(contains(@*,'lead') and contains(text(),'Para el resto del año, también destacan las presentaciones'))]/p/parent::*/../../../@class"
#aplicamos uan palabra clave y vemos las clases de los padres
summary=notice_home.xpath(contains)
len(summary), summary[:10]

(4, ['row article-wrapper', 'col-9', '', 'container pt-5 pb-5'])

In [ ]:
contains="//body//div[contains(@*,'article-wrapper')]//descendant::*"
#filtramos por la clase 'article-wrapper' 
summary=notice_home.xpath(contains)
len(summary), summary[:10]

(71,
 [<Element div at 0x7fa934d9afb0>,
  <Element div at 0x7fa934d9c590>,
  <Element div at 0x7fa934d9cb30>,
  <Element a at 0x7fa934e9b2f0>,
  <Element img at 0x7fa934ea74d0>,
  <Element ul at 0x7fa934ea7050>,
  <Element li at 0x7fa934ea7410>,
  <Element p at 0x7fa934dab4d0>,
  <Element li at 0x7fa934e188f0>,
  <Element a at 0x7fa934e18bf0>])

In [ ]:
contains="//body//div[contains(@*,'article-wrapper')]//div/p/text()"
#alicamos el filtro
summary=notice_home.xpath(contains)
len(summary), summary[:10]

(7,
 ['El teatro cuenta con un calendario de más de 30 eventos para los próximos meses, que incluyen música, danza y festivales ',
  'Los planes culturales volvieron a la presencialidad con fuerza. En el caso del Teatro Mayor Julio Mario Santo Domingo, se ha traducido en un incremento de más de 30% en los asistentes desde que se levantaron las restricciones de movilidad.',
  'La agenda está enriquecida con más de 30 eventos entre lo que resta de febrero y abril, que incluyen, música, danza y por supuesto, teatro. Con los que se espera aumentar aún más la asistencia de los espectadores.',
  '"La pandemia es un ejemplo de cómo las audiencias se volcaron a la oferta cultural digital, llegando a números que en su momento eran inimaginables. Esperamos que ahora ese consumo se vea reflejado en lo presencial, pues el público que compra entradas para los eventos culturales es un patrocinador fundamental", destacó Ramiro Osorio, director del Teatro Mayor Julio Mario Santo Domingo.',
  'Para el 